<a href="https://colab.research.google.com/github/amedyukhina/biomassters/blob/main/biomassters_dataprep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile requirements.txt

pandas==1.3.5
tqdm==4.64.0
numpy==1.21.6
scikit-image==0.18.3
matplotlib==3.2.2
rasterio==1.2.0
boto3==1.26.16

Writing requirements.txt


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 5.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1


In [3]:
from google.colab import drive
import os
import rasterio
import pandas as pd
from tqdm import tqdm
import numpy as np
from skimage import io

import boto3
from botocore import UNSIGNED
from botocore.config import Config

from cachetools import cached, TTLCache

import warnings
from tqdm.notebook import tqdm
import shutil

# Our rasters contain no geolocation info, so silence this warning from rasterio
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Prepare list of chip IDs

In [5]:
feature_path = 'gdrive/MyDrive/biomassters/info/features_metadata.csv'
data_path = 'gdrive/MyDrive/biomassters/data/'
train_img_dir = 'train_features'
test_img_dir = 'test_features'
label_dir = 'train_agbm'
ntrain = 200
nval = 20
nmeans = 100

In [6]:
df = pd.read_csv(feature_path)

In [7]:
df_pred = df[df['split'] == 'test'].reset_index(drop=True)
pred_ids = np.unique(df_pred['chip_id'])
len(df_pred), len(pred_ids)

(63348, 2773)

In [8]:
df = df[df['split'] == 'train'].reset_index(drop=True)
all_ids = np.unique(df['chip_id'])
len(df), len(all_ids)

(189078, 8689)

In [9]:
np.random.seed(42)
np.random.shuffle(all_ids)

In [10]:
train_ids = all_ids[:ntrain]
val_ids = all_ids[ntrain:ntrain+nval]

### Set up data loading

In [11]:
BUCKET_NAME = 'drivendata-competition-biomassters-public-us'
os.environ["AWS_NO_SIGN_REQUEST"] = 'YES'
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [12]:
cache = TTLCache(maxsize=1000, ttl=86400)

In [13]:
# @cached(cache)
# def get_image(fn):
#   obj = s3.get_object(Bucket=BUCKET_NAME, Key=fn)
#   with rasterio.open(obj['Body']) as src:
#     img = src.read()
#   return img

@cached(cache)
def get_image(fn):
    img = io.imread(fn)
    if img.shape[-1] < 20:
        img = np.moveaxis(img, -1, 0)
    return img

### Calculate mean and std of the dataset + save images to gdrive

In [14]:
# # training data
# os.makedirs(data_path + train_img_dir, exist_ok=True)
# os.makedirs(data_path + label_dir, exist_ok=True)
# imgs_s1 = []
# imgs_s2 = []
# it = 0
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     for chip_id in tqdm(train_ids):
#         for month in range(12):
#             img = get_image(os.path.join(train_img_dir, 
#                                         rf"{chip_id}_S1_{month:02d}.tif")).astype(np.float32)   
#             io.imsave(os.path.join(data_path, train_img_dir, 
#                                   rf"{chip_id}_S1_{month:02d}.tif"), 
#                       img.astype(np.float32))   
#             if it < nmeans:                     
#                 imgs_s1.append(img)

#             try:
#                 img = get_image(os.path.join(train_img_dir, 
#                                             rf"{chip_id}_S2_{month:02d}.tif")).astype(np.float32)
#                 io.imsave(os.path.join(data_path, train_img_dir, 
#                                       rf"{chip_id}_S2_{month:02d}.tif"), 
#                           img.astype(np.uint16))   
#                 if it < nmeans:                         
#                     imgs_s2.append(img)
#             except:
#                 pass

#         label = get_image(os.path.join(label_dir, 
#                                         rf"{chip_id}_agbm.tif"))
#         io.imsave(os.path.join(data_path, label_dir, 
#                                 rf"{chip_id}_agbm.tif"), 
#                   label.astype(np.uint16))
#         it += 1

# imgs_s1 = np.array(imgs_s1)
# imgs_s2 = np.array(imgs_s2)

In [18]:

imgs_s1 = []
imgs_s2 = []
it = 0
for chip_id in tqdm(train_ids):
    for month in range(12):
        img = get_image(os.path.join(data_path + train_img_dir, 
                                    rf"{chip_id}_S1_{month:02d}.tif")).astype(np.float32)   
        if it < nmeans:                     
            imgs_s1.append(img)

        try:
            img = get_image(os.path.join(data_path + train_img_dir, 
                                        rf"{chip_id}_S2_{month:02d}.tif")).astype(np.float32)
            if it < nmeans:                         
                imgs_s2.append(img)
        except:
            pass
    it += 1

imgs_s1 = np.array(imgs_s1)
imgs_s2 = np.array(imgs_s2)

  0%|          | 0/20 [00:00<?, ?it/s]

In [46]:
imgs = imgs_s1
MEANS = np.array([imgs[:,i][np.where(imgs[:,i] != -9999)].mean() for i in range(imgs.shape[1])])
STDS = np.array([imgs[j,i].std() for j in range(imgs.shape[0]) 
  for i in range(imgs.shape[1])]).reshape((imgs.shape[:2])).mean(0)
np.save(data_path + 's1_mean_std.npy', [MEANS, STDS])
MEANS, STDS

(array([-11.454795 , -18.013327 , -11.5029745, -18.152424 ], dtype=float32),
 array([2.2330995, 2.964378 , 2.0271204, 2.6701245], dtype=float32))

In [47]:
imgs = imgs_s2
MEANS = np.array([imgs[:,i].mean() for i in range(imgs.shape[1])])
STDS = np.array([imgs[j,i].std() for j in range(imgs.shape[0]) 
  for i in range(imgs.shape[1])]).reshape((imgs.shape[:2])).mean(0)
np.save(data_path + 's2_mean_std.npy', [MEANS, STDS])
MEANS, STDS

(array([1642.8818  , 1621.115   , 1598.5988  , 1918.794   , 2490.597   ,
        2604.4023  , 2753.5115  , 2696.2065  , 1048.9395  ,  710.3785  ,
          21.115498], dtype=float32),
 array([1026.1313  ,  994.45984 , 1044.3563  , 1076.0272  , 1126.5492  ,
        1129.6202  , 1215.7893  , 1130.656   ,  521.7572  ,  426.1423  ,
          11.847122], dtype=float32))

In [ ]:
# validation data
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for chip_id in tqdm(val_ids):
        for month in range(12):
            img = get_image(os.path.join(train_img_dir, 
                                        rf"{chip_id}_S1_{month:02d}.tif")).astype(np.float32)   
            io.imsave(os.path.join(data_path, train_img_dir, 
                                  rf"{chip_id}_S1_{month:02d}.tif"), 
                      img.astype(np.float32))   

            try:
                img = get_image(os.path.join(train_img_dir, 
                                            rf"{chip_id}_S2_{month:02d}.tif")).astype(np.float32)
                io.imsave(os.path.join(data_path, train_img_dir, 
                                      rf"{chip_id}_S2_{month:02d}.tif"), 
                          img.astype(np.uint16))   
            except:
                pass

        label = get_image(os.path.join(label_dir, 
                                        rf"{chip_id}_agbm.tif"))
        io.imsave(os.path.join(data_path, label_dir, 
                                rf"{chip_id}_agbm.tif"), 
                  label.astype(np.uint16))


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# inference data
os.makedirs(data_path + test_img_dir, exist_ok=True)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for chip_id in tqdm(pred_ids):
        for month in range(12):
            img = get_image(os.path.join(test_img_dir, 
                                        rf"{chip_id}_S1_{month:02d}.tif")).astype(np.float32)   
            io.imsave(os.path.join(data_path, test_img_dir, 
                                  rf"{chip_id}_S1_{month:02d}.tif"), 
                      img.astype(np.float32))   

            try:
                img = get_image(os.path.join(test_img_dir, 
                                            rf"{chip_id}_S2_{month:02d}.tif")).astype(np.float32)
                io.imsave(os.path.join(data_path, test_img_dir, 
                                      rf"{chip_id}_S2_{month:02d}.tif"), 
                          img.astype(np.uint16))   
            except:
                pass

  0%|          | 0/2773 [00:00<?, ?it/s]